In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymysql
from sqlalchemy import create_engine
import time
from sklearn.preprocessing import StandardScaler, normalize, RobustScaler, MinMaxScaler, FunctionTransformer
from scipy.stats import boxcox
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import scipy.cluster.hierarchy as hc
from sklearn.cluster import AgglomerativeClustering
from haversine import haversine, Unit

In [109]:
df1 = pd.read_csv('/home/subin/다운로드/여피/0308 업무-20230310T030110Z-001/0308 업무/complex_20230308 (사본).csv', header=None)
df1.columns = ['old_addr_idx', 'complex_name', 'property_type','시도', '시군구',
               '읍면동','리','road_name','road_code','admin_dong_code','postal_code','jibun_main','jibun_sub','updated']
df1.drop(['road_name','road_code','admin_dong_code','postal_code','jibun_main','jibun_sub', 'updated'], axis=1, inplace=True)

df1['시군구'] = df1['시군구'].str.split().str[0]
df1 = df1[df1['property_type']!='officetel']
df1.drop(['property_type'], axis=1, inplace=True)

In [110]:
df2 = pd.read_csv('/home/subin/다운로드/여피/0308 업무-20230310T030110Z-001/0308 업무/property_20230308.csv', header=None)
df2.columns = ['pk','old_addr_idx','dong','ho','official_price','net_leasable_area','updated']

In [111]:
app_apt = df2.merge(df1, on='old_addr_idx')
pd.options.display.max_columns = None
app_apt['평수'] = app_apt['net_leasable_area'] / 3.3
app_apt.drop(['net_leasable_area'], axis=1, inplace=True)
app_apt.replace('\\N', np.nan, inplace=True)
app_apt['시군구'] = app_apt['시군구'].fillna('세종특별자치시')

In [112]:
app_apt

,pk,old_addr_idx,dong,ho,official_price,updated,complex_name,시도,시군구,읍면동,리,평수
0,00000001_0001_0001,1111010100_00004_00007_apartment,단일,101,1249000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,63.978788
1,00000001_0001_0002,1111010100_00004_00007_apartment,단일,102,1076000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,63.396970
2,00000001_0001_0003,1111010100_00004_00007_apartment,단일,201,1140000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,62.996970
3,00000001_0001_0004,1111010100_00004_00007_apartment,단일,202,1249000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,65.178788
4,00000001_0001_0005,1111010100_00004_00007_apartment,단일,301,1332000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,66.463636
...,...,...,...,...,...,...,...,...,...,...,...,...
14496524,20432545_0001_0012,2647010200_01288_00036_apartment,A,404,43100000,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,NaN,5.539394
14496525,20432545_0001_0013,2647010200_01288_00036_apartment,A,502,43300000,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,NaN,5.630303
14496526,20432545_0001_0014,2647010200_01288_00036_apartment,A,504,42400000,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,NaN,5.539394
14496527,20432545_0001_0015,2647010200_01288_00036_apartment,A,501,42400000,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,NaN,5.539394


In [113]:
daejeon = app_apt[app_apt['시도']=='대전광역시']
daejeon

,pk,old_addr_idx,dong,ho,official_price,updated,complex_name,시도,시군구,읍면동,리,평수
3032783,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,42.500000
3032784,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,41.733333
3032785,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032786,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032787,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
...,...,...,...,...,...,...,...,...,...,...,...,...
14493165,20431058_0001_0014,3014011000_00233_00043_apartment,단일,802,215000000,2022-01-01,더퍼스트뷰,대전광역시,중구,대사동,NaN,25.747364
14493166,20431058_0001_0015,3014011000_00233_00043_apartment,단일,901,215000000,2022-01-01,더퍼스트뷰,대전광역시,중구,대사동,NaN,25.747364
14493167,20431058_0001_0016,3014011000_00233_00043_apartment,단일,902,215000000,2022-01-01,더퍼스트뷰,대전광역시,중구,대사동,NaN,25.747364
14493168,20431058_0001_0017,3014011000_00233_00043_apartment,단일,1001,250000000,2022-01-01,더퍼스트뷰,대전광역시,중구,대사동,NaN,22.255485


In [117]:
pd.set_option('display.max_rows', 10)

In [116]:
pd.set_option('display.max_rows', None)
daejeon[daejeon['old_addr_idx']=='3014010200_00160_00001_apartment']

,pk,old_addr_idx,dong,ho,official_price,updated,complex_name,시도,시군구,읍면동,리,평수
3032783,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,42.500000
3032784,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,41.733333
3032785,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032786,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032787,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032788,00010892_0001_0006,3014010200_00160_00001_apartment,단일,206,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032789,00010892_0001_0007,3014010200_00160_00001_apartment,단일,207,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.448485
3032790,00010892_0001_0008,3014010200_00160_00001_apartment,단일,208,60500000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,26.090909
3032791,00010892_0001_0009,3014010200_00160_00001_apartment,단일,301,92100000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,40.654545
3032792,00010892_0001_0010,3014010200_00160_00001_apartment,단일,302,90800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,39.881818


In [114]:
df3[df3['old_addr_idx']=='3014010200_00160_00001_apartment']

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동
23340,3014010200_00160_00001_apartment,82000000,24.683333,4,2020-05-21,NaN,2020-05-21,NaN,NaN,NaN
44278,3014010200_00160_00001_apartment,75000000,26.878788,3,2018-03-09,NaN,2018-03-09,NaN,NaN,NaN
44279,3014010200_00160_00001_apartment,60000000,24.683333,4,2018-04-02,NaN,2018-04-02,NaN,NaN,NaN
44280,3014010200_00160_00001_apartment,27500000,18.166667,6,2018-07-26,NaN,2018-07-26,NaN,NaN,NaN
59196,3014010200_00160_00001_apartment,68000000,24.683333,4,2021-01-18,NaN,2021-01-18,NaN,NaN,NaN
59197,3014010200_00160_00001_apartment,86000000,24.683333,4,2021-08-03,NaN,2021-08-03,NaN,NaN,NaN
82567,3014010200_00160_00001_apartment,82000000,26.878788,3,2019-06-28,NaN,2019-06-28,NaN,NaN,NaN


In [75]:
df3 = pd.read_csv('/home/subin/다운로드/여피/0315 업무/trade_history_2018-2023_edit.csv', header=None)
df3.columns = ['old_addr_idx', '거래날짜', 'actual', '면적', 'floor', '모름', '거래방식', '모름2', '모름3', '시군구', '주소', '읍면동', 'complex_name', '모름4', '모름5', 'property_type']
df3 = df3.drop_duplicates() 
df3['area'] = df3['면적']/3.3
df3.fillna("NONE", inplace=True)
df3 = df3[~df3['property_type'].str.contains('officetel')]
df3 = df3[['old_addr_idx', 'actual', 'area', 'floor', '거래날짜', 'complex_name', '거래날짜', '시군구', '주소', '읍면동']]

<ipython-input-75-3d74f6f0beb1>:1: DtypeWarning: Columns (5,6,8,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('/home/subin/다운로드/여피/0315 업무/trade_history_2018-2023_edit.csv', header=None)


In [77]:
df3 = df3.replace('NONE', np.nan)
df3.isnull().sum()

old_addr_idx          0
actual                0
area                  0
floor                 0
거래날짜                  0
complex_name    2136478
거래날짜                  0
시군구             2137611
주소              2136478
읍면동             2136478
dtype: int64

In [78]:
df3

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동
0,3023010200_00034_00000_apartment,99000000,17.890909,2,2020-02-28,NaN,2020-02-28,NaN,NaN,NaN
1,3023010200_00027_00007_apartment,70000000,17.048485,5,2020-02-03,NaN,2020-02-03,NaN,NaN,NaN
2,3023010200_00027_00007_apartment,66000000,17.048485,1,2020-03-22,NaN,2020-03-22,NaN,NaN,NaN
3,3023010200_00027_00007_apartment,77000000,17.048485,3,2020-04-29,NaN,2020-04-29,NaN,NaN,NaN
4,3023010200_00027_00007_apartment,70000000,17.048485,5,2020-05-25,NaN,2020-05-25,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2203363,5011010400_01034_00001_apartment,280000000,25.436970,5,2022-09-16,이도월드하우스,2022-09-16,제주 제주시,1034-1,이도이동
2203364,5011010400_00777_00000_apartment,580000000,12.027273,4,2022-09-27,주공3,2022-09-27,제주 제주시,777,이도이동
2203365,5013010400_02161_00000_apartment,317000000,25.704212,2,2022-09-01,호근동코아루오션뷰,2022-09-01,제주 서귀포시,2161,호근동
2203366,5013010500_00147_00000_apartment,214000000,12.027273,1,2022-09-01,주공1,2022-09-01,제주 서귀포시,147,동홍동


In [10]:
address = df1[['old_addr_idx', 'sido', 'sigungu', 'eubmyeondong', 'li']]

In [12]:
address = address.drop_duplicates()

In [13]:
address

,old_addr_idx,sido,sigungu,eubmyeondong,li
0,1111010100_00001_00000_apartment,서울특별시,종로구,청운동,NaN
1,1111010100_00003_00000_apartment,서울특별시,종로구,청운동,NaN
2,1111010100_00003_00150_apartment,서울특별시,종로구,청운동,NaN
3,1111010100_00004_00001_apartment,서울특별시,종로구,청운동,NaN
4,1111010100_00004_00003_apartment,서울특별시,종로구,청운동,NaN
...,...,...,...,...,...
320920,5013032022_00048_00003_apartment,제주특별자치도,서귀포시,표선면,하천리
320921,5013032022_02167_00000_apartment,제주특별자치도,서귀포시,표선면,하천리
320922,5013032023_02008_00000_apartment,제주특별자치도,서귀포시,표선면,성읍리
320923,5013032024_02521_00005_apartment,제주특별자치도,서귀포시,표선면,가시리


In [23]:
sidolist = address['sido'].unique().tolist()

In [105]:
k=5
each = address[address['sido']==sidolist[k]]
each

,old_addr_idx,sido,sigungu,eubmyeondong,li
178887,3011010100_00035_00003_apartment,대전광역시,동구,원동,NaN
178888,3011010200_00012_00005_apartment,대전광역시,동구,인동,NaN
178890,3011010200_00014_00001_apartment,대전광역시,동구,인동,NaN
178891,3011010200_00018_00001_apartment,대전광역시,동구,인동,NaN
178892,3011010200_00029_00002_apartment,대전광역시,동구,인동,NaN
...,...,...,...,...,...
184117,3023012600_00264_00046_apartment,대전광역시,대덕구,신탄진동,NaN
184118,3023012600_00264_00069_apartment,대전광역시,대덕구,신탄진동,NaN
184119,3023012600_00295_00293_apartment,대전광역시,대덕구,신탄진동,NaN
184120,3023012600_00451_00000_apartment,대전광역시,대덕구,신탄진동,NaN


In [30]:
check_act = act_name.merge(address, on='old_addr_idx', how='outer')
check_act

,old_addr_idx,sido,sigungu,eubmyeondong,li
0,3023010200_00034_00000_apartment,대전광역시,대덕구,대화동,NaN
1,3023010200_00027_00007_apartment,대전광역시,대덕구,대화동,NaN
2,3023010200_00016_00002_apartment,대전광역시,대덕구,대화동,NaN
3,3023010200_00040_00067_apartment,대전광역시,대덕구,대화동,NaN
4,3023010200_00031_00036_apartment,대전광역시,대덕구,대화동,NaN
...,...,...,...,...,...
306017,5013032022_00048_00003_apartment,제주특별자치도,서귀포시,표선면,하천리
306018,5013032022_02167_00000_apartment,제주특별자치도,서귀포시,표선면,하천리
306019,5013032023_02008_00000_apartment,제주특별자치도,서귀포시,표선면,성읍리
306020,5013032024_02521_00005_apartment,제주특별자치도,서귀포시,표선면,가시리


In [79]:
not_exist_off = check_act[check_act['sido'].isnull()==True]
not_exist_off

,old_addr_idx,sido,sigungu,eubmyeondong,li
34949,1121510300_00680_00000_apartment,NaN,NaN,NaN,NaN
34956,1129013300_00150_00027_apartment,NaN,NaN,NaN,NaN
34962,1141011000_00149_00000_apartment,NaN,NaN,NaN,NaN
34972,1168010100_00902_00000_apartment,NaN,NaN,NaN,NaN
34982,2620010100_00176_00000_apartment,NaN,NaN,NaN,NaN
...,...,...,...,...,...
35660,1159010700_01160_00000_apartment,NaN,NaN,NaN,NaN
35663,2917010100_00836_00000_apartment,NaN,NaN,NaN,NaN
35667,4146310200_00661_00000_apartment,NaN,NaN,NaN,NaN
35679,4874025021_00053_00000_apartment,NaN,NaN,NaN,NaN


In [82]:
not_exist_off_list = not_exist_off['old_addr_idx'].unique().tolist()

In [88]:
not_exist_off_df = df3[df3['old_addr_idx']==not_exist_off_list[0]]

i=1
while i<len(not_exist_off_list):
    for_concat = df3[df3['old_addr_idx']==not_exist_off_list[i]]
    not_exist_off_df = pd.concat([not_exist_off_df, for_concat])
    i=i+1
not_exist_off_df

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동
2184966,1121510300_00680_00000_apartment,1600000000,35.135394,24,2022-10-23,e편한세상광진그랜드파크,2022-10-23,NaN,680,구의동
2184993,1129013300_00150_00027_apartment,193000000,3.893939,10,2022-10-03,갤럭시더정릉,2022-10-03,NaN,150-27,정릉동
2185033,1141011000_00149_00000_apartment,1240000000,18.136364,2,2022-10-29,e편한세상신촌3단지,2022-10-29,서울 서대문구,149,북아현동
2200659,1141011000_00149_00000_apartment,1100000000,18.136364,17,2022-11-05,e편한세상신촌2단지,2022-11-05,서울 서대문구,149,북아현동
2185118,1168010100_00902_00000_apartment,3000000000,25.731818,8,2022-10-06,강남센트럴아이파크,2022-10-06,"서울 강남구, 서울 송파구",902,역삼동
...,...,...,...,...,...,...,...,...,...,...
2200689,1159010700_01160_00000_apartment,895000000,18.179030,12,2022-11-27,사당롯데캐슬골든포레,2022-11-27,서울 동작구,1160,사당동
2202760,2917010100_00836_00000_apartment,870000000,33.269697,13,2022-10-30,제일풍경채센트럴파크1단지,2022-10-30,광주 북구,836,중흥동
2202775,4146310200_00661_00000_apartment,800000000,25.751818,33,2022-10-04,기흥역더샵,2022-10-04,NaN,661,구갈동
2203333,4874025021_00053_00000_apartment,64000000,18.163636,5,2022-09-26,도원,2022-09-26,경남 창녕군,53,창녕읍 송현리


In [93]:
not_exist_off_df['시도'] = not_exist_off_df['시군구'].str.split(' ').str[0]

In [95]:
not_exist_off_df

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동,시도
2184966,1121510300_00680_00000_apartment,1600000000,35.135394,24,2022-10-23,e편한세상광진그랜드파크,2022-10-23,NaN,680,구의동,NaN
2184993,1129013300_00150_00027_apartment,193000000,3.893939,10,2022-10-03,갤럭시더정릉,2022-10-03,NaN,150-27,정릉동,NaN
2185033,1141011000_00149_00000_apartment,1240000000,18.136364,2,2022-10-29,e편한세상신촌3단지,2022-10-29,서울 서대문구,149,북아현동,서울
2200659,1141011000_00149_00000_apartment,1100000000,18.136364,17,2022-11-05,e편한세상신촌2단지,2022-11-05,서울 서대문구,149,북아현동,서울
2185118,1168010100_00902_00000_apartment,3000000000,25.731818,8,2022-10-06,강남센트럴아이파크,2022-10-06,"서울 강남구, 서울 송파구",902,역삼동,서울
...,...,...,...,...,...,...,...,...,...,...,...
2200689,1159010700_01160_00000_apartment,895000000,18.179030,12,2022-11-27,사당롯데캐슬골든포레,2022-11-27,서울 동작구,1160,사당동,서울
2202760,2917010100_00836_00000_apartment,870000000,33.269697,13,2022-10-30,제일풍경채센트럴파크1단지,2022-10-30,광주 북구,836,중흥동,광주
2202775,4146310200_00661_00000_apartment,800000000,25.751818,33,2022-10-04,기흥역더샵,2022-10-04,NaN,661,구갈동,NaN
2203333,4874025021_00053_00000_apartment,64000000,18.163636,5,2022-09-26,도원,2022-09-26,경남 창녕군,53,창녕읍 송현리,경남


In [91]:
pd.set_option('display.max_rows', 10)

In [96]:
not_exist_off_df.to_csv('공시지가에 없는 실거래가 아파트 코드.csv')

In [98]:
read = pd.read_csv('/home/subin/MyGitProject/hsj-14/공시지가에 없는 실거래가 아파트 코드.csv')
read

,Unnamed: 0,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜.1,시군구,주소,읍면동,시도
0,2184966,1121510300_00680_00000_apartment,1600000000,35.135394,24,2022-10-23,e편한세상광진그랜드파크,2022-10-23,NaN,680,구의동,서울
1,2184993,1129013300_00150_00027_apartment,193000000,3.893939,10,2022-10-03,갤럭시더정릉,2022-10-03,NaN,150-27,정릉동,서울
2,2185033,1141011000_00149_00000_apartment,1240000000,18.136364,2,2022-10-29,e편한세상신촌3단지,2022-10-29,서울 서대문구,149,북아현동,서울
3,2200659,1141011000_00149_00000_apartment,1100000000,18.136364,17,2022-11-05,e편한세상신촌2단지,2022-11-05,서울 서대문구,149,북아현동,서울
4,2185118,1168010100_00902_00000_apartment,3000000000,25.731818,8,2022-10-06,강남센트럴아이파크,2022-10-06,"서울 강남구, 서울 송파구",902,역삼동,서울
...,...,...,...,...,...,...,...,...,...,...,...,...
90,2200689,1159010700_01160_00000_apartment,895000000,18.179030,12,2022-11-27,사당롯데캐슬골든포레,2022-11-27,서울 동작구,1160,사당동,서울
91,2202760,2917010100_00836_00000_apartment,870000000,33.269697,13,2022-10-30,제일풍경채센트럴파크1단지,2022-10-30,광주 북구,836,중흥동,광주
92,2202775,4146310200_00661_00000_apartment,800000000,25.751818,33,2022-10-04,기흥역더샵,2022-10-04,NaN,661,구갈동,경기
93,2203333,4874025021_00053_00000_apartment,64000000,18.163636,5,2022-09-26,도원,2022-09-26,경남 창녕군,53,창녕읍 송현리,경남


In [99]:
read[read['시도']=='대전']

,Unnamed: 0,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜.1,시군구,주소,읍면동,시도
59,2192458,3014010300_00001_00095_apartment,620000000,25.755091,26,2022-08-03,목동더샵리슈빌,2022-08-03,대전 중구,1-95,목동,대전
66,2197556,3014010200_00379_00001_apartment,229120000,14.999606,7,2022-06-25,에이스퀘어,2022-06-25,대전 중구,379-1,선화동,대전


In [31]:
check_act.isnull().sum()

old_addr_idx         0
sido                94
sigungu            493
eubmyeondong     11003
li              266222
dtype: int64

In [27]:
check_each.notnull().sum()

old_addr_idx    35683
sido             7529
sigungu          7529
eubmyeondong     7525
li                100
dtype: int64

In [15]:
address['sido'].value_counts()

서울특별시      105668
경기도         82801
인천광역시       26573
부산광역시       24201
경상남도         9310
대구광역시        9197
경상북도         9099
충청남도         6277
제주특별자치도      5778
대전광역시        5098
울산광역시        4684
충청북도         4086
전라북도         3549
강원도          3430
전라남도         3350
광주광역시        2419
세종특별자치시       408
Name: sido, dtype: int64

In [56]:
act_code = df3[['old_addr_idx']].drop_duplicates()

In [57]:
act_code

,old_addr_idx
0,3023010200_00034_00000_apartment
1,3023010200_00027_00007_apartment
9,3023010200_00016_00002_apartment
35,3023010200_00040_00067_apartment
41,3023010200_00031_00036_apartment
...,...
2203326,4833010200_00565_00001_apartment
2203333,4874025021_00053_00000_apartment
2203338,4882025021_00048_00012_apartment
2203346,4886037021_00601_00000_apartment


In [48]:
act_code_withName = df3[['old_addr_idx', 'complex_name']].drop_duplicates()

In [50]:
act_code_withName = act_code_withName.replace('NONE', np.nan)

In [51]:
act_code_withName

,old_addr_idx,complex_name
0,3023010200_00034_00000_apartment,NaN
1,3023010200_00027_00007_apartment,NaN
9,3023010200_00016_00002_apartment,NaN
35,3023010200_00040_00067_apartment,NaN
41,3023010200_00031_00036_apartment,NaN
...,...,...
2203363,5011010400_01034_00001_apartment,이도월드하우스
2203364,5011010400_00777_00000_apartment,주공3
2203365,5013010400_02161_00000_apartment,호근동코아루오션뷰
2203366,5013010500_00147_00000_apartment,주공1


In [54]:
39624-35683

3941

In [52]:
act_name = act_name.replace('NONE', np.nan)
act_name = act_code_withName.dropna(subset='complex_name').drop_duplicates()

In [53]:
act_name

,old_addr_idx,complex_name
2184937,1111017500_00202_00003_apartment,종로중흥S클래스
2184938,1111017500_00318_00002_apartment,삼전솔하임4차
2184939,1111017500_00296_00019_apartment,삼전솔하임2차
2184940,1114015400_00145_00001_apartment,오렌지카운티을지로
2184941,1114016200_00432_01666_apartment,남산FORESTHOUSE
...,...,...
2203363,5011010400_01034_00001_apartment,이도월드하우스
2203364,5011010400_00777_00000_apartment,주공3
2203365,5013010400_02161_00000_apartment,호근동코아루오션뷰
2203366,5013010500_00147_00000_apartment,주공1


In [65]:
a = act_name.drop('complex_name', axis=1)

In [72]:
a

,old_addr_idx
2184937,1111017500_00202_00003_apartment
2184938,1111017500_00318_00002_apartment
2184939,1111017500_00296_00019_apartment
2184940,1114015400_00145_00001_apartment
2184941,1114016200_00432_01666_apartment
...,...
2203363,5011010400_01034_00001_apartment
2203364,5011010400_00777_00000_apartment
2203365,5013010400_02161_00000_apartment
2203366,5013010500_00147_00000_apartment


In [71]:
a.drop_duplicates()

,old_addr_idx
2184937,1111017500_00202_00003_apartment
2184938,1111017500_00318_00002_apartment
2184939,1111017500_00296_00019_apartment
2184940,1114015400_00145_00001_apartment
2184941,1114016200_00432_01666_apartment
...,...
2203363,5011010400_01034_00001_apartment
2203364,5011010400_00777_00000_apartment
2203365,5013010400_02161_00000_apartment
2203366,5013010500_00147_00000_apartment


In [68]:
b = a[a.duplicated()==True]

In [70]:
b

,old_addr_idx
2184990,1129010300_00609_00001_apartment
2185122,1168010300_00012_00000_apartment
2185196,2632010100_00810_00000_apartment
2185330,2814010300_00154_00000_apartment
2186474,4831010100_00224_00000_apartment
2188676,2811011800_00027_00107_apartment
2188695,2818510200_00353_00000_apartment
2188778,3171025032_01329_00002_apartment
2189013,4217010100_00794_00000_apartment
2189218,4623010600_01665_00000_apartment


In [73]:
c = b.merge(act_name, on='old_addr_idx', how='left')

In [74]:
c

,old_addr_idx,complex_name
0,1129010300_00609_00001_apartment,한신
1,1129010300_00609_00001_apartment,한진(609-1)
2,1168010300_00012_00000_apartment,삼익대청아파트
3,1168010300_00012_00000_apartment,성원대치2단지아파트
4,2632010100_00810_00000_apartment,금곡8단지주공
...,...,...
69,1159010700_00105_00000_apartment,극동(105)
70,1159010700_00105_00000_apartment,사당우성2
71,4146510100_00664_00000_apartment,동아
72,4146510100_00664_00000_apartment,풍림아파트


In [69]:
b.shape

(30, 1)

In [58]:
act_code_merge_name = act_code.merge(act_name, on='old_addr_idx', how='left')

In [59]:
act_code_merge_name

,old_addr_idx,complex_name
0,3023010200_00034_00000_apartment,NaN
1,3023010200_00027_00007_apartment,NaN
2,3023010200_00016_00002_apartment,금성백조
3,3023010200_00040_00067_apartment,대화
4,3023010200_00031_00036_apartment,두레
...,...,...
35708,4833010200_00565_00001_apartment,양산신도시 벽산블루밍
35709,4874025021_00053_00000_apartment,도원
35710,4882025021_00048_00012_apartment,그네빌리지
35711,4886037021_00601_00000_apartment,MH미르젠아파트


In [19]:
check_act = act_name.merge(address, on='old_addr_idx', how='left')

In [20]:
check_act

,old_addr_idx,sido,sigungu,eubmyeondong,li
0,3023010200_00034_00000_apartment,대전광역시,대덕구,대화동,NaN
1,3023010200_00027_00007_apartment,대전광역시,대덕구,대화동,NaN
2,3023010200_00016_00002_apartment,대전광역시,대덕구,대화동,NaN
3,3023010200_00040_00067_apartment,대전광역시,대덕구,대화동,NaN
4,3023010200_00031_00036_apartment,대전광역시,대덕구,대화동,NaN
...,...,...,...,...,...
35678,4833010200_00565_00001_apartment,경상남도,양산시,남부동,NaN
35679,4874025021_00053_00000_apartment,NaN,NaN,NaN,NaN
35680,4882025021_00048_00012_apartment,경상남도,고성군,고성읍,성내리
35681,4886037021_00601_00000_apartment,NaN,NaN,NaN,NaN


In [21]:
check_act.isnull().sum()

old_addr_idx        0
sido               94
sigungu           106
eubmyeondong      214
li              29790
dtype: int64

In [22]:
check_act['sido'].value_counts()

서울특별시      7529
경기도        5974
부산광역시      3846
경상남도       3097
경상북도       2372
인천광역시      1840
대구광역시      1686
울산광역시      1328
전라북도       1102
전라남도       1046
충청남도       1039
강원도        1037
광주광역시       974
충청북도        974
제주특별자치도     921
대전광역시       812
세종특별자치시      12
Name: sido, dtype: int64